## Chapter 12

### Tensorflow

* 강력한 수치 계산용 라이브러리. 특히 대규모 머신러닝에 잘 맞도록 튜닝되어있음.
  * numpy와 비슷한 구조, GPU지원.
  * 분산 컴퓨팅 지원
  * JIT 컴파일러 포함. 속도를 높이고 메모리 사용량을 줄이기 위해 계산을 최적화하고, 이를 위해 계산 그래프를 추출해 최적화하여 효율적으로 실행.
  * 한 환경에서 텐서플로 모델을 훈련하고 다른 환경에서 실행가능.
  * 자동 미분, 고성능 optimizer 제공.
* 위의 핵심 기술을 기반으로 매우 많은 기능 제공
  * <code>tf.keras</code>가 가장 중요
  * 데이터 적재 및 전처리 연산(<code>tf.data, tf.io</code>등), 이미지 처리 연산(<code>tf.image</code>), 신호 처리 연산(<code>tf.signal</code>) 등의 기능을 제공
* 많은 연산이 커널(kernel)이라고 부르는 여러 구현을 가짐.
* 각 커널은 CPU, GPU, TPU(텐서 연산 장치, Tensor processing unit)와 같은 특정 장치에 맞춰 만들어짐.
  * GPU는 계산을 작은 단위로 나눠 여러 GPU thread에서 병렬로 실행하므로 속도를 향상시킴.
  * TPU는 gPU보다 더 빠름.
* 시각화를 위해 TensorBoard, 텐서플로 제품화를 위한 라이브러리 모음인 TFX(https://tensorflow.org/tfx) (데이터 시각화, 전처리, 모델 분석, serving 등 포함), 사전훈련된 신경망을 다운받을 수 있는 텐서플로 허브(https://github.com/tensorflow/models) 등 사용가능.

### Numpy처럼 Tensorflow 사용하기

* Tensor는 한 연산에서 다른 연산으로 흐름.
* Tensor는 <code>ndarray</code>와 비슷. (즉, 일반적으로 다차원 배열)
  * 스칼라 값(42 등)도 가질 수 있음.

In [1]:
# 텐서 생성
# 두 개의 행과 세 개의 열을 가진 실수 행렬

import tensorflow as tf

tf.constant([[1., 2., 3.], [4., 5., 6.]])

d:\pythonenv\mlenv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\pythonenv\mlenv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
d:\pythonenv\mlenv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [2]:
# 스칼라 생성

tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

* ndarray와 마찬가지로 <code>tf.Tensor</code>는 크기와 데이터 타입(dtype)을 가짐.

In [3]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

In [4]:
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
# index 참조

t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

* 모든 종류의 텐서 연산 가능

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

> * t + 10 : tf.add(t, 10)
>   * -, * 등도 지원
> * @ : 행렬 곱셈. tf.matmul()과 동일
* 기본적 수학 연산 (<code>tf.add(), tf.multiply(), tf.square(), tf.exp(), tf.sqrt()</code>등)과 numpy의 대부분의 연산들(<code>tf.reshape(), tf.squeeze(), tf.tile()</code>)을 제공.
  * 단, 일부는 조금씩 다름(<code>tf.reduce_mean()</code> == <code>np.mean()</code> 등)

* keras API에서는 <code>keras.backend</code>에 자체적인 저수준 API를 포함.
  * <code>square(), exp(), sqrt()</code>등. 이들은 상응하는 텐서플로 연산을 호출.
  * 다른 케라스 구현에 적용할 수 있는 코드를 작성하려면 이러한 케라스 함수를 사용해야함.

In [11]:
from tensorflow import keras

K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

* ndarray <-> Tensor 간 변환이 가능. 또한, ndarray에 텐서플로 연산을 사용할 수 있고, Tensor에 넘파이 연산을 사용할 수 있음.

In [12]:
import numpy as np

a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [13]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [14]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [15]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

> numpy는 64bit 정밀도를 사용하고 tensorflow는 32bit 정밀도를 사용하므로 numpy 배열로 tensor를 만들려면 <code>dtype=tf.float32</code>로 지정해야함.

* 타입 변환이 가능하나, 성능을 크게 감소시킬 수 있고 타입이 자동으로 변환되면 사용자가 눈치채지 못할 수 있으므로 텐서플로에서는 타입 변환을 자동으로 수행하지 않음.

In [16]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

> InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [ ]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

> InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

In [ ]:
# 단, type 변환이 필요한 경우 바꿀 수는 있음. (tf.cast())

t2 = tf.constant(40, dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

* tf.Tensor는 변경이 불가능한 객체. 즉, 일반적인 텐서로는 역전파로 변경해야 하는 신경망의 가중치를 구현할 수 없음.
* 이를 해결하기 위해 <code>tf.Variable</code> 사용.
* tf.Tensor와 비슷하게 동작함. 대신, <code>assign()</code>을 이용해 변수값을 바꿀 수 있음.
  * <code>assign_add(), assign_sub()</code>를 이용해 변수값 증가, 감소도 가능.
  * 또한, 원소의 <code>assign()</code>이나 <code>scatter_update(), scatter_nd_update()</code>로 개별 원소 또는 슬라이스를 수정할 수 있음.

In [ ]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

In [ ]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [ ]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [ ]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [ ]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [ ]:
v.scatter_nd_update(indices=[[0, 0], [1,2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

> Keras에서는 <code>add_weight()</code>로 변수 생성을 대신하기 때문에 변수를 직접 만드는 일은 잘 없고, 변수 업데이트도 optimizer가 하기 때문에 수동으로 업데이트하는 일도 거의 없음.

* 텐서플로는 다른 데이터 구조도 지원.
  * 희소 텐서(<code>tf.SparseTensor</code>) : 대부분 0으로 채워진 텐서. <code>tf.sparse</code>에서 희소 텐서 연산 제공.
  * 텐서 배열(<code>tf.TensorArray</code>) : 텐서의 리스트. 고정된 길이를 가지나 동적으로 바꿀 수 있음. 리스트 내의 모든 텐서는 크기와 데이터 타입이 동일해야함.
  * 래그드 텐서(ragged tensor)(<code>tf.RaggedTensor</code>) : 리스트의 리스트. 텐서에 포함된 값은 동일한 데이터 타입을 가져야 하지만, 리스트의 길이는 다를 수 있음. <code>tf.ragged</code>에서 ragged tensor를 위한 연산 제공
  * 문자열 텐서(<code>tf.string</code>) : byte 문자열을 나타냄. 유니코드 문자열을 사용해 만들면 자동으로 UTF-8로 인코딩되고(ex. b"caf\xc3\xa9"), <code>tf.int32</code> 텐서를 이용해 유니코드 코드 포인트(ex. [99,97, 192, 233])로 표현할 수 있음. <code>tf.string**s**</code>는 byte 문자열 <-> 유니코드 문자열 텐서 사이의 변환을 위한  연산 제공.
  * set : 일반적인 텐서로 나타남. (ex. <code>tf.constant([[1,2], [3,4]]) 는 두 집합 {1,2}, {3,4}를 나타냄</code>). 각 집함은 텐서의 마지막 축의 벡터에 의해 표현. <code>tf.sets</code>사용
  * queue : 단계별로 텐서 저장. <code>tf.queue</code>에 포함.

### 사영자 정의 모델 및 훈련 알고리즘

* 훈련 세트에 잡음 데이터가 섞여 있다면 평균 제곱 오차는 큰 오차에 너무 과한 벌칙을 가하기 때문에 부정확한 모델이 만들어지고, 평균 절댓값 오차는 이상치에 관대해 훈련이 수렴되기까지 오래 걸리고 정밀하게 훈련되지 않음.
* 이런 경우, 후버 손실(오차가 임곗값 $\sigma$(주로 1)보다 작을 때는 이차함수. 이보다 클 때는 선형 함수.)을 사용하는 것이 좋음.
* 후버 손실은 공식 keras API에서는 지원하지 않음.
  * <code>tf.keras</code>에서는 <code>keras.losses.Huber</code> 클래스가 있지만, 여기서는 직접 구현
  * 레이블과 예측을 매개변수로 받는 함수를 만들고, 텐서플로 연산을 사용해 샘플의 손실 계산.

In [17]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

> * 성능을 위해서는 위처럼 벡터화하여 구현해야 하고, 텐서플로 그래프의 장점을 활용하려면 텐서플로 연산만 사용해야함.

* 전체 손실의 평균이 아닌 샘플마다 하나의 손실을 담은 텐서를 반환해야 케라스가 클래스 가중치나 샘플 가중치를 적용할 수 있음.

In [ ]:
model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train, [...])

* 훈련하는 동안 매 배치마다 huber_fn()을 호출해 손실을 계산하고 이를 사용해 경사 하강법 수행.

* 사용자 정의 요소를 가진 모델을 저장할 때는 그 함수 이름을 저장하므로 이상 없이 저장됨.
* 단, 불러올 때는 함수 이름과 실제 함수를 매핑한 딕셔너리를 전달해야 함.(즉, 사용자 정의 객체를 포함한 모델을 로드할 때 그 이름과 객체를 매핑해야 함.)

In [ ]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5", custom_objects={"huber_fn", huber_fn})

* 위의 huber_fn()은 -1과 1사이의 오차는 작은 것으로 간주함. 이러한 기준을 변경하려면 매개변수를 받을 수 있는 함수를 만들어 사용할 수 있음.

In [ ]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")

* 이러한 모델을 저장할 때는 <code>threshold</code>값이 저장되지 않으므로 모델을 로드할 때 이 값을 지정해야 함.

In [ ]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5", custom_objects={"huber_fn": create_huber(2.0)})

* 또는 <code>keras.losses.Loss</code>를 상속하고 <code>get_config()</code>를 구현해 해결할 수 있음.

In [19]:
class HuberLoss(keras.losses.Loss):
    # 생성자. **kwargs로 받은 매개변수 값을 부모 클래스 생성자에 전달(손실 함수 name과 개별 샘플의 손실을 모으기 위해 사용할 reduction 알고리즘).
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    # 레이블과 예측을 받고 모든 샘플의 손실을 계산하여 반환.
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    # 하이퍼파라미터 이름과 같이 매핑된 딕셔너리 반환.
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [ ]:
model.compile(loss=HuberLoss(2.), optimizer="nadam")

* 모델 저장 시에는 임곗값도 함께 저장됨.
* 모델 로드 시에는 클래스 이름과 클래스 자체를 매핑해줘야 함.

In [ ]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5", custom_objects={"HuberLoss": HuberLoss})

* 모델 저장 시 케라스는 get_config()를 호출하여 반환된 설정을 JSON 형태로 저장하고, 모델을 로드하면 HuberLoss클래스의 from_config()(기본 손실 클래스 <code>Loss</code>에 구현되어 있음)를 호출하여 생성자에게 **config 매개변수를 전달해 클래스의 인스턴스를 만듦.

* 손실, 규제, 제한, 초기화, 지표, 활성화 함수, 층, 모델과 같은 대부분의 기능들은 유사한 방법으로 커스터마이징할 수 있음.
* 적절한 입력과 출력을 가진 간단한 함수를 작성하는 것으로 가능.

In [20]:
# 사용자 정의 활성화 함수(keras.activations.softplus(), tf.nn.softplus()와 같은 기능)
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

# 사용자 정의 glorot 초기화(keras.initializers.glorot_normal()과 같은 기능)
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# 사용자 정의 l1 규제(keras.regularizers.l1(0.01)과 같은 기능)
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# 양수인 가중치만 남기는 사용자 정의 제한(keras.constraints.nonneg(), tf.nn.relu()와 같은 기능)
def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

* 매개변수는 사용자 정의하려는 함수의 종류에 따라 다름.
* 사용자 정의 함수는 보통의 함수와 동일하게 사용 가능.

In [21]:
layer = keras.layers.Dense(30, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

> * 활성화 함수는 Dense층의 출력에 적용되고 다음 층에 그 결과가 전달.
> * 층의 가중치는 초기화 함수에서 반환된 값으로 초기화.
> * 훈련 스텝마다 가중치가 규제 함수에 전달되어 규제 손실을 계산하고, 전체 손실에 추가되어 훈련을 위한 최종 손실을 만듦.
> * 제한 함수가 훈련 스테마다 호출되어 가중치들을 제한한 가중치 값으로 바꿈.
* 함수가 모델과 함께 저장해야할 파라미터를 가지고 있다면 적절한 클래스를 상속해야 함.
  * <code>keras.regularizers.Regularizer, keras.constraints.Constraint, keras.initializer.Initizlizer, keras.layers.Layer</code>

In [22]:
# factor 하이퍼파라미터를 저장하는 l1 규제를 위한 클래스.
# 부모 클래스에 생성자와 get_config()가 정의되어 있으므로 호출할 필요가 없음.
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

* 손실, (활성화 함수를 포함하여) 층, 모델은 <code>call()</code>을 구현해야 함.
* 규제, 초기화, 제한은 <code>\_\_call\_\_()</code>를 구현해야 함.

* 사용자 정의 지표는 약간 다름
  * 손실과 지표는 개념적으로는 동일
  * 손실 : 모델 훈련을 위해 경사 하강법에서 사용하므로 미분 가능해야 하고 모든 곳에서 gradient가 0이 아니어야 함. 또한, 사람이 쉽게 이해할 수 없어도 괜찮음.
  * 지표 : 모델을 평가할 때 사용하고(정확도 등), 사람이 이해하기 쉬워야 함. 미분불가능하거나 모든 곳에서 gradient가 0이어도 괜찮음.
* 대부분의 경우, 사용자 지표 함수를 만드는 것은 사용자 손실 함수를 만드는 것과 동일.
  * 앞의 huber_fn()은 지표로 사용해도 잘 동작함(실제로 후버 함수는 지표로 잘 사용하지 않지만). 모델 저장 시에서 동일하게 함수의 이름만 저장.

In [ ]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

* 훈련 동안 각 배치에 대해 지표를 계산하고, epoch가 시작할 때 부터 평균을 기록.
* 이러한 방식이 대부분의 요구 사항에 맞지만 모두 그런 것은 아님.
  * 이진 분류기의 경우, 진짜 앙셩 개수/양성 예측 개수(=진짜 양성+거짓 양성)로 계산되는 정밀도를 사용하는데, 만약 첫 번째 배치에서 5개의 양성 예측을 만들고 그 중 4개가 맞았다면 정밀도는 80%가 되는데 두 번째 배치에서 3개의 양성 예측을 만들었지만 모두 틀렸다면 정밀도는 0%가 됨. 그렇다면 정밀도의 평균은 40%이지만, 실제로 두 배치를 합해서 보면 8개의 양성 예측 중에 4개가 진짜 양성이므로 전체 정밀도는 50%가 되어야 함.
  * 이러한 경우 <code>keras.metrics.Precision</code>클래스를 사용(진짜 양성 개수와 거짓 양성 개수를 기록하고 필요할 때 정밀도를 계산할 수 있는 객체)

In [23]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [24]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

> * 첫 번째 배치를 처리할 때는 정밀도가 80%이고, 두 번째 배치를 처리하고 난 뒤에는 정밀도가 50%가 됨. 즉, 두 번째 배치 처리 시 두 번째 배치에 대한 정밀도가 아니라 지금까지의 전체 정밀도를 계산함.
* 배치마다 점진적으로 업데이트되기 때문에 스트리밍 지표(streaming metric) 또는 상태가 있는 지표(stateful metric)라고 함.
* <code>result()</code>를 호출하여 현재 지표값을 얻을 수 있음.
* <code>variables</code>속성을 사용해 진짜 양성과 거짓 양성을 기록한 변수를 확인할 수 있음. <code>reset_states()</code>를 사용해 이 변수를 초기화할 수 있음.

In [25]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [26]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [27]:
precision.reset_states()

* 스트리밍 지표를 만들려면 <code>keras.metrics.Metric</code>클래스를 상속.

In [28]:
# 전체 후버 손실과 지금까지 처리한 샘플 수를 기록하는 클래스.
# 결과값 요청 시 평균 후버 손실 반환

class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)  # dtype과 같은 기본 매개변수 처리
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        # add_weight로 여러 배치에 걸쳐 지표의 상태를 기록하기 위한 변수 만듦. 수동으로도 만들 수 있음. keras는 속성으로 만들어진 모든 tf.Variable을 관리.
        self.total = self.add_weight("total", initializer="zeros")  # 후버 손실의 합
        self.count = self.add_weight("count", initializer="zeros")  # 지금까지 처리한 샘플 수
    # HuberMetric을 함수처럼 사용할 때 호출됨. 배치의 레이블과 예측, (샘플 가중치)를 바탕으로 변수를 업데이트
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    # 최종 결과 계산 및 반환. 여기서는 모든 샘플에 대한 평균 후버 손실값을 반환함.
    def result(self):
        return self.total / self.count
    # threshold 변수를 모델과 함께 처리함.
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}
    
# reset_status()는 모든 변수를 0.0으로 초기화. 필요하다면 이 함수도 오버라이딩 가능.

* 텐서플로에 없는 특이한 층을 만들려면 사용자 정의 층을 만들 수 있음.
* 동일한 층 블럭이 여러 번 반복되는 네트워크를 만들 때 각 층 블럭을 하나의 층으로 다루는 것이 좋음.
* 가중치가 필요 없는(<code>keras.layers.Flatten, keras.layers.ReLU</code>와 같이) 사용자 정의 층은 파이썬 함수를 만든 뒤 <code>keras.layers.Lambda</code> 층으로 감싸면 됨.

In [29]:
# 입력에 지수 함수를 적용하는 층

exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

* 사용자 정의 층은 Sequential API, 함수형 API, 서브클래싱 API에서 보통 층처럼 동일하게 사용할 수 있음.
* 또한, 활성화 함수로 사용할 수도 있음
  * <code>activation=tf.exp</code> / <code>activation=keras.activations.exponential</code> / <code>activation="exponential"</code>을 사용할 수도 있음.
  * 지수 함수는 회귀 모델에서 예측값의 스케일이 매우 다를 때 출력층에 사용됨(ex. 0.001, 10, 1.000 등)

* 가중치를 가진(상태가 있는) 층을 만들려면 <code>keras.layers.Layer</code>를 상속해야함.

In [30]:
# Dense층의 간소화 버전 구현
class MyDense(keras.layers.Layer):
    # 하이퍼파라미터(여기서는 units, activation)를 매개변수로 받음
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)  # 부모 생성자를 통해 **kwargs를 넘겨 input_shape, trainable, name과 같은 기본 매개변수를 처리할 수 있음.
        self.units = units
        self.activation = keras.activations.get(activation) # 객체나 "relu"같이 문자열을 받아 적절한 활성화 함수로 바꿔 속성으로 저장.
        
    # 가중치마다 add_weight()를 호출해 층의 변수를 만듦. 층이 처음 사용될 때 호출됨.
    # 이 시점에선 keras가 층의 입력 크기를 알고 있으므로 build()의 입력으로 크기를 전달(보통 input_shape라고 부르지만, 여기서는 배치 차원이 포함되었으므로 batch_input_size로 사용).
    # 연결 가중치를 만들려면 이전 층의 뉴런 개수를 알아야 함. 이 크기는 입력의 마지막 차원 크기에 해당
    # build()에는 반드시 끝에 부모의 build()를 호출해야 keras가 층이 만들어졌다는 것을 인식함.
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel",
            shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal"
        )
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)    # 마지막에 호출해야함.
        
    # 층에 필요한 연산을 수행. X와 층의 kernel을 행렬곱셈하고 편향을 더한 뒤, 그 결과에 활성화 함수를 적용함. 이 값이 층의 출력이 됨.
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    
    # 층의 출력 크기 반환. 마지막 차원을 제외하고 입력과 크기가 같음. 마지막 차원은 이 층의 뉴런의 개수.
    # tf.keras에서의 크기는 tf.TensorShape클래스의 객체로, as_list()를 이용해 파이썬 리스트로 변환할 수 있음
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    
    # 활성화 함수의 전체 설정 저장.
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, "activation": keras.activations.serialize(self.activation)}

> * 동적인 층을 제외하고 tf.keras가 자동으로 출력 크기를 추측할 수 있다면 compute_output_shape()를 생략할 수 있음. 다른 keras 구현에서는 이 메서드가 필수이거나, 출력 크기가 입력 크기와 동일하다고 가정.
* Concatenate층과 같은 여러 입력을 받는 층은 call()내에 모든 입력이 포함된 튜플을 매개변수 값으로 전달해야 하고, compute_output_shape()의 매개변수도 각 입력의 배치 크기를 담은 튜플이어야 함.
* 여러 출력을 가진 층을 만들려면 call()이 출력 list를 반환해야 하고, compute_output_shape()는 출력마다 하나씩 배치 출력 크기의 list를 반환해야함.

In [ ]:
# 두 개의 입력과 세 개의 출력을 만드는 층

class MyMultiLayer(keras.layers.Layer):
   def call(self, X):
       X1, X2 = X
       return [X1+X2, X1*X2, X1/X2] 
   
   def compute_output_shape(self, batch_input_shape):
       b1, b2 = batch_input_shape
       return [b1, b1, b1]

> * 다른 일반적인 층처럼 사용할 수 있지만, 함수형 API와 서브클래싱 API에만 사용가능.

* 훈련과 테스트에서 다르게 동작하는 층이 필요하다면(<code>Dropout, BatchNormalization</code>과 같이) call()내에 training 매개변수를 초가하여 훈련인지 테스트인지 결정해야 함.

In [31]:
# 훈련하는 동안에는 규제 목적으로 가우스 잡음을 추가하고 테스트 시에는 아무것도 하지 않는 층
# keras.layers.gaussianNoise와 동일

class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
        
    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X
        
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

* 사용자 정의 모델은 <code>keras.Model</code>을 상속하여 생성자에서 층과 변수를 만들고, 모델이 해야 할 작업을 call()에 구현.
* ex) 스킵 연결이 있는 사용자 정의 잔차 블록(ResidualBlock) 층을 가진 모델
  * 입력이 첫 번째 완전 연결 층을 통과해, 두 개의 완전 연결 층과 스킵 연결로 구성된 잔차 블록(출력에 입력을 더함)으로 전달.
  * 이후, 동일한 잔차 블록에 세번 더 통과시킴.
  * 이후, 두 번째 잔차 블록을 지나 마지막 출력이 완전 연결된 출력 층에 전달.
> * 실제로 사용되는 구조는 아님. 단순한 예시.

In [34]:
# 동일한 잔차 블록을 여러 개 만들어야 하므로 ResidualBlock층을 만듦.
# 다른 층을 포함하는 특별한 형태.
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        # keras가 알아서 추적해야 할 객체(여기에서는 층)가 담긴 hidden 속성을 감지하고, 필요한 변수를 이 층의 변수 리스트에 추가.
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu", kernel_initializer="he_normal")
                       for _ in range(n_layers)]
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs+Z
    
# 서브클래싱 API를 이용해 모델 정의
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)
        
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)